In [ ]:
!pip install BeautifulSoup4
!pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# ambil berita detik
def getBeritaDetik(url):
    B = {}
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # ambil elemen-elemen berita
    B['judul'] = soup.find('h1', {'class': 'detail__title'}).text.replace('\n', "").strip()
    B['tanggal'] = soup.find('div', {'class': 'detail__date'}).text.replace('\n', "").strip()
    B['author'] = soup.find('div', {'class': 'detail__author'}).text.replace('\n', "").strip()
    berita = soup.find('div', {'class': 'detail__body-text'})
    text_berita = berita.text

    # bersihkan isi berita
    blah = berita.find_all("table")
    for x in blah:
        text_berita = text_berita.replace(x.text, "").replace('\n', "").strip()

    B['berita'] = text_berita
    return B

In [ ]:
import requests
from bs4 import BeautifulSoup

def indexBerita(tanggal, jumlahHalaman):
    daftarBerita = []
    halaman = 0

    for halaman in range(0, jumlahHalaman):
        halaman = halaman + 1

        base_url = 'https://news.detik.com/berita/indeks?page=' + str(halaman) + "&date=" + tanggal
        # print(base_url)

        # Request URL dan BeautifulSoup Parser
        r = requests.get(base_url)
        soup = BeautifulSoup(r.text, "html.parser")

        berita_container = soup.find('div', {'id': 'indeks-container'})
        berita = berita_container.find_all('article')

        for item in berita:
            x = item.find('a', href=True)
            berita = getBeritaDetik(x['href'])
            daftarBerita.append(berita)

    return daftarBerita

In [ ]:
# Contoh penggunaan
tanggal = '02/02/2021'
jumlahHalaman = 1

list_berita = indexBerita(tanggal, jumlahHalaman)

# Ubah ke DataFrame
df = pd.DataFrame(list_berita)
print(df.columns)
print(df)

Index(['judul', 'tanggal', 'author', 'berita'], dtype='object')
                                                judul  \
0   Gencar Senayan Suarakan Jaminan WNI Aman di Te...   
1   KKB Serang Warga-Aparat Bikin Para Raider 501 ...   
2   Polemik Lampu Jalan Padam di Pekanbaru Imbas N...   
3   5 Nelayan Aceh yang Dipulangkan dari India Pos...   
4   Syuting Picu Kerumunan Warga, Manajemen Sinetr...   
5   KPK Terus Pantau Pengadaan hingga Distribusi V...   
6   Wakil Ketua Komisi XI Imbau Masyarakat Waspada...   
7       Kepulauan Talaud Sulut Diguncang Gempa 2 Kali   
8   RS Pelni Luruskan Viral Poster Pendaftaran-Har...   
9   Beri Santunan Anak Yatim, Bamsoet Ajak Doa Ber...   
10  Pemprov DKI soal Pilkada 2024: Kami Ikuti UU y...   
11  Bacok Pencuri hingga Tewas, Pria di Bima Jadi ...   
12  Pertamina Perketat Pencegahan COVID-19 di Unit...   
13  5 Hal Terungkap di Kasus Asabri Kuras Duit Neg...   
14  PPSU di Jakpus Nyaris Jadi Korban Curanmor, Pe...   
15  Respons Marah Marzuk

In [ ]:
df.to_csv('berita_detik.csv', index=False)

In [ ]:
df.to_excel('berita_detik.xlsx', index=False)